# Data Dictionary Workbook

This notebook contains examples of using Python to transform metadata among data dictionary formats.


## AWS Glue Catalog to/from RAML Data Types
[AWS Glue Catalog](https://docs.aws.amazon.com/glue/latest/dg/catalog-and-crawler.html) stores structural metadata that describes data assets hosted on AWS Native Services, such as S3, Redshift Spectrum, Athena, EMR, and SageMaker.  For our purposes Glue Catalog contains tablular data, although it does have features to support JSON and even XML.

[RAML](https://raml.org/) is a modeling language used to describe the functionality and data content of REST APIs.  RAML is built on top of YAML, a superset of JSON that supports hierarchical structured data.  Both YAML and JSON are human-readable and -editable.   YAML especially so, using indents to indicate hierarchical levels and dashes to identify list items, it is essentialy a formal method of outlining.

AWS Glue Catalog is extensible and customizable using 'Parameters' comprised of key/value pairs.  Unfortunately, AWS Console provides only rudimentary support for viewing the Glue Catalog, and even less for manually editing it.  So, since RAML has more friendly tools for for editing metadata, a function to import RAML into Glue Catalog has been included this workbook.   Basic structural data can be exported from Glue, manually enhanced, and imported back into Glue whence it is available for exports elsewhere.


In [1]:
## export a Glue Table to a RAML Data Type file
#  (requires AWS connectivity and authorization)
import sys
if './python' not in sys.path: sys.path.append('./python')
    
from glue_functions import export_glue_to_raml

# sample Glue database and table
glue_database_name = "dtl-prd-tdsx"
glue_table_name = "detailed_top_data_source"
output_path = f'file://data/{glue_table_name}.raml'

raml = export_glue_to_raml( glue_database_name, glue_table_name, output_path )

#print(raml)


Output to file 'file://data/detailed_top_data_source.raml'


In [2]:
## import a RAML Data Type file into Glue Catalog
#  (requires AWS connectivity and authorization)
import sys
if './python' not in sys.path: sys.path.append('./python')

# sample Glue database and table
glue_database_name = "dtl-dev-raml-import"
glue_table_name = "detailed_top_data_source"
raml_file = f'file://data/{glue_table_name}.raml'
glue_sd_json = f'file://data/{glue_table_name}.glue_sd.json'

from glue_functions import get_file
raml = get_file(raml_file)

from glue_functions import import_raml_to_glue
glue_sd_columns = import_raml_to_glue( glue_database_name, glue_table_name, raml, glue_sd_json )
#print(glue_sd_columns)

from glue_functions import crup_glue_table
crup_glue_table( glue_database_name, glue_table_name, glue_sd_columns )


Output to file 'file://data/detailed_top_data_source.glue_sd.json'
Glue Table detailed_top_data_source updated in Database dtl-dev-raml-import


{'ResponseMetadata': {'RequestId': 'c38a45a4-6791-4136-8ef7-f5fe6510d881',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 18 Nov 2022 13:41:21 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c38a45a4-6791-4136-8ef7-f5fe6510d881'},
  'RetryAttempts': 0}}

## CSV to/from RAML

MS Excel is a popular tool for manually editing tabular information, including data dictionaries.   Stored as a CSV file, 'semi-structured' data created or modified in Excel can be validated and imported to more structured formats.

### FiscalData Data Dictionaries & Meta Objects
The Department of the Treasury and the Bureau of the Fiscal Service created FiscalData to consolidate federal financial data into one easy-to-use website. 
https://fiscaldata.treasury.gov/

FiscalData datasets are available for download in multiple machine-readable formats, and feature customizable data previews. Each dataset is complete with comprehensive metadata to explain when the data is updated, what each row of the data represents, and descriptions for each dataset and data table.

Structural metadata is provided as 
1. Data Dictionaries downloadable as CSV files
2. Meta objects embedded in API responses



In [3]:
## export a FiscalData CSV Data Dictionary file to a RAML Data Type

import sys
if './python' not in sys.path: sys.path.append('./python')

from dd_functions import export_fddatadict_to_raml
#help(export_fddatadict_to_raml)

# example from 'Download Data Dictionary' link on https://fiscaldata.treasury.gov/datasets/treasury-offset-program/federal-collections  ...
csv_input_path = "data/Treasury Offset Program Data Dictionary.csv"
raml_output_path = csv_input_path.replace('.csv','.raml').replace(' ','_').lower()

export_fddatadict_to_raml( csv_input_path, raml_output_path )


Output to file 'data/treasury_offset_program_data_dictionary.raml'


In [4]:
## import a RAML Data Type file into Glue Catalog
#  (requires AWS connectivity and authorization)
import sys
if './python' not in sys.path: sys.path.append('./python')

# sample Glue database and table
glue_database_name = "dtl-dev-raml-import"
glue_table_name = "treasury_offset_program_data_dictionary"
raml_file = f'file://data/{glue_table_name}.raml'
glue_sd_json = f'file://data/{glue_table_name}.glue_sd.json'

from glue_functions import get_file
raml = get_file(raml_file)

from glue_functions import import_raml_to_glue
glue_sd_columns = import_raml_to_glue( glue_database_name, glue_table_name, raml, glue_sd_json )
#print(glue_sd_columns)

from glue_functions import crup_glue_table
crup_glue_table( glue_database_name, glue_table_name, glue_sd_columns )


Output to file 'file://data/treasury_offset_program_data_dictionary.glue_sd.json'
Glue Table treasury_offset_program_data_dictionary updated in Database dtl-dev-raml-import


{'ResponseMetadata': {'RequestId': '3da223da-a7c9-4a60-a9ef-bb0b8faf8b8c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 18 Nov 2022 13:43:57 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3da223da-a7c9-4a60-a9ef-bb0b8faf8b8c'},
  'RetryAttempts': 0}}

In [6]:
# example from https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/debt/top/top_federal?fields=record_date,creditor_agency_nm,creditor_agency_id,agency_type,agency_site
# meta object copied from response ...
import sys
if './python' not in sys.path: sys.path.append('./python')

from dd_functions import export_fdmeta_to_raml

meta = {
  "count": 100,
  "labels": {
    "record_date": "Record Date",
    "creditor_agency_nm": "Creditor Agency Name",
    "creditor_agency_id": "Agency ID",
    "agency_type": "Agency Site Type Code",
    "agency_site": "Agency Site ID"
  },
  "dataTypes": {
    "record_date": "DATE",
    "creditor_agency_nm": "STRING",
    "creditor_agency_id": "STRING",
    "agency_type": "STRING",
    "agency_site": "STRING"
  },
  "dataFormats": {
    "record_date": "YYYY-MM-DD",
    "creditor_agency_nm": "String",
    "creditor_agency_id": "String",
    "agency_type": "String",
    "agency_site": "String"
  },
  "total-count": 161141,
  "total-pages": 1612
}

export_fdmeta_to_raml( meta )

#%RAML 1.0

title: Import from FiscalData Meta Object
types:
  record_date:
    type: date
    displayName: Record Date
    format: YYYY-MM-DD
  creditor_agency_nm:
    type: string
    displayName: Creditor Agency Name
    format: String
  creditor_agency_id:
    type: string
    displayName: Agency ID
    format: String
  agency_type:
    type: string
    displayName: Agency Site Type Code
    format: String
  agency_site:
    type: string
    displayName: Agency Site ID
    format: String

